# Part 2: Postgres

In [1]:
HOST = "10.77.240.11"
USER = "postgres"
PW=
DB="postgres"

In [2]:
import psycopg

def connect():
    
    url = "dbname='{}' user='{}' host={} port=5432 password={}".format(DB, USER, HOST, PW)
    connection = psycopg.connect(url)

    return connection

### 1. Test Connection

In [3]:
import psycopg

connection = connect()

print(connection)

<psycopg.Connection [IDLE] (host=10.77.240.11 database=postgres) at 0x7f1f0557fac0>


### 2. Create the table (and drop it if it already exists)

In [4]:
import psycopg

connection = connect()
print(connection)

drop = 'DROP TABLE reservations'

create = '''CREATE TABLE reservations (
    res_id smallint NOT NULL,
    cust_id smallint NOT NULL,
    prp_nm varchar(50) NOT NULL,
    prp_ch varchar(50) NOT NULL,
    adr_line_1 varchar(50) NOT NULL,
    adr_line_2 varchar(16) NOT NULL,
    city varchar(50) NOT NULL,
    state char(2) NOT NULL,
    postal_cd char(5) NOT NULL,
    lat float NOT NULL,
    long float NOT NULL,
    cnt_code char(2) NOT NULL,
    arr_date date NOT NULL,
    dep_date date NOT NULL,
    pmt_amt numeric(5,2) NOT NULL,
    PRIMARY KEY (res_id)
) '''

print(drop)
print(create)

try:
    cursor = connection.cursor()
    cursor.execute(drop)
    print('Dropped person table')
    
    cursor = connection.cursor()
    cursor.execute(create)
    print('Created person table')
    
    connection.commit() # commit the changes 
except Exception as e:
    print("Error occurred while creating person table:", e)

<psycopg.Connection [IDLE] (host=10.77.240.11 database=postgres) at 0x7f1f0557f0a0>
DROP TABLE reservations
CREATE TABLE reservations (
    res_id smallint NOT NULL,
    cust_id smallint NOT NULL,
    prp_nm varchar(50) NOT NULL,
    prp_ch varchar(50) NOT NULL,
    adr_line_1 varchar(50) NOT NULL,
    adr_line_2 varchar(16) NOT NULL,
    city varchar(50) NOT NULL,
    state char(2) NOT NULL,
    postal_cd char(5) NOT NULL,
    lat float NOT NULL,
    long float NOT NULL,
    cnt_code char(2) NOT NULL,
    arr_date date NOT NULL,
    dep_date date NOT NULL,
    pmt_amt numeric(5,2) NOT NULL,
    PRIMARY KEY (res_id)
) 
Dropped person table
Created person table


### 3. Insert records into table

In [5]:
import psycopg
import random
from datetime import datetime
from faker import Faker

connection = connect()

psql = "INSERT INTO reservations VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

cnt_codes = ['TZ', 'BR', 'MC', 'ER', 'MV', 'GY', 'VU', 'BE', 'LU', 'ME', 'MR', 'VA', 'MY', 'CL', 'GA', 
             'JO', 'SA', 'LK', 'CV', 'MY', 'CR', 'ME', 'CG', 'HN', 'AG', 'LY', 'BE', 'CD', 'CG', 'PT']

records = []
for i in range(100):
    fake = Faker()
    
    # Generate random address
    adr_line_1, address = fake.address().split(sep="\n")
    adr_line_2 = ""
    if adr_line_1.split()[-2] in ["Apt.", "Suite"]:      
        adr_line_2 = adr_line_1.split()[-2] + " " + adr_line_1.split()[-1]
        adr_line_1 = adr_line_1.split(sep=adr_line_1.split()[-2])[0][:-1]

    city_state_zip = address.split(sep=",")
    while len(city_state_zip) == 1:
        city_state_zip = fake.address().split(sep="\n")[1].split(sep=",")
    state, postal_cd = city_state_zip[1].split()
    arr_date = fake.date_object()
    
    record = [
        i,
        random.randint(0, 49),
        fake.company(),
        fake.company(),
        adr_line_1,
        adr_line_2,
        city_state_zip[0],
        state,
        postal_cd,
        fake.coordinate(),
        fake.coordinate(),
        random.choice(cnt_codes),
        arr_date,
        fake.date_between(start_date = arr_date),
        round(random.random() * 999, 2)
    ]
    records.append(record)

try:
    cursor = connection.cursor()
    
    for rec in records:
        cursor.execute(psql, rec)
    
    connection.commit()
    print("100 records written into reservations table")
        
except Exception as e:
    print("Failed to insert record into reservations table:", e)

100 records written into reservations table
